In [1]:
import numpy as np
from numpy import sys
np.set_printoptions(threshold=sys.maxsize)
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score
from sklearn import svm
import pandas as pd
import sys
import os
import matplotlib
matplotlib.use("TkAgg")
from matplotlib import pyplot as plt   
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg #, NavigationToolbar2TkAgg
from matplotlib.figure import Figure

import csv

In [2]:
#'22.04.07_acryl_concentration.csv' : Concentration classification_dark chamber
#'22.04.08_Acryl_Stress.csv' : Stress classification_dark chamber
#'22.05.19_Dominant.csv' : Dominant classification_dark chamber

#1-healthy, 2-acid,-3,heat,4-heat-acid // 5,6,7,8,9-5~100x10^4cells/mL // 10-green algae, 11-red algae, 12-brown algae, 13- even mixture

data_file = '22.04.07_acryl_concentration.csv'
accuracy_threshold= -1

In [3]:
df = pd.read_csv(data_file)

In [4]:
y =df.loc[:,'Name'].values
x =df.drop(['Name'],axis=1)

X=np.asarray(x)
Y=np.asarray(y)


In [7]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(X,Y, test_size=0.3, random_state=4)

In [28]:
Y_test.shape

(41,)

In [1]:
for gam in (0.001, 0.01, 0.1, 1, 10, 100):
    for c in (0.001, 0.01, 0.1, 1, 10, 100):
        for ker in ('poly', 'linear', 'rbf'):
            #Define SVM parameters
            clf = svm.SVC(kernel=ker, gamma=gam, C=c)
            # Run the classifier for ALL PCs to predict the Test data
            y_pred = clf.fit(X_train, Y_train).predict(X_test)
            accuracy = accuracy_score(Y_test, y_pred)
            if accuracy > accuracy_threshold:
                print(ker, gam, c, accuracy)

NameError: name 'svm' is not defined

In [2]:
from sklearn import svm

classifier = svm.SVC(kernel='linear',gamma=0.01,C=100)
classifier.fit(X_train,Y_train)

y_predict=classifier.predict(X_test)

NameError: name 'X_train' is not defined

In [9]:
from sklearn.metrics import classification_report

print(classification_report(Y_test,y_predict))

              precision    recall  f1-score   support

       Brown       1.00      0.73      0.84        11
        Even       0.45      0.71      0.56         7
       Green       1.00      0.59      0.74        17
         Red       0.50      1.00      0.67         6

    accuracy                           0.71        41
   macro avg       0.74      0.76      0.70        41
weighted avg       0.83      0.71      0.73        41



In [10]:
from sklearn.metrics import confusion_matrix

#Generate the confusion matrix
cf_matrix = confusion_matrix(Y_test, y_predict)

print(cf_matrix)

[[ 8  2  0  1]
 [ 0  5  0  2]
 [ 0  4 10  3]
 [ 0  0  0  6]]


In [ ]:
import seaborn as sns

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['Green','Red','Brown','Even'])
ax.yaxis.set_ticklabels(['Green','Red','Brown','Even'])

## Display the visualization of the Confusion Matrix.
plt.show()